#### Load the dataset.

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors  import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import RobustScaler

df = pd.read_csv('../dataset/preprocessed-sam-dataset.csv', sep='|',
                 dtype = {'CZ': 'float32', 'FZ': 'float32', 'Fp1': 'float32', 'F3': 'float32',
                          'FC1': 'float32', 'FC5': 'float32', 'FT9': 'float32', 'T7': 'float32',
                          'CP5': 'float32', 'P3': 'float32', 'P7': 'float32', 'PO9': 'float32',
                          'PZ': 'float32', 'O2': 'float32', 'P4': 'float32', 'CP6': 'float32',
                          'FT10': 'float32', 'FC6': 'float32', 'F8': 'float32', 'Fp2': 'float32',
                          'Scale': 'int8'})


#### Display the dataset size.

In [13]:
df.shape

(140800, 21)

#### Train the model and display metrics.

In [14]:
X = df.drop('Scale', axis = 1)
y = df['Scale']

X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size = 0.3, random_state = 42)

scaler = RobustScaler()
scaled_trainX = scaler.fit_transform(X_train)
scaled_testX = scaler.transform(X_test)

model = KNeighborsClassifier(n_jobs = 4)
model.fit(scaled_trainX, y_train)
y_pred = model.predict(scaled_testX)

print("Train Accuracy: {:.2f} %".format(model.score(scaled_trainX, y_train) * 100))
print("Test Accuracy: {:.2f} %".format(accuracy_score(y_test, y_pred) * 100))
print('\n')
print("Classifiction Report")
print(classification_report(y_test, y_pred, zero_division = 0))


Train Accuracy: 36.74 %
Test Accuracy: 11.34 %


Classifiction Report
              precision    recall  f1-score   support

           0       0.09      0.20      0.13      3820
           1       0.12      0.24      0.16      3843
           2       0.11      0.16      0.13      3924
           3       0.10      0.09      0.10      3870
           4       0.10      0.09      0.09      3808
           5       0.10      0.07      0.09      3823
           6       0.11      0.06      0.08      3820
           7       0.10      0.05      0.07      3890
           8       0.12      0.07      0.09      3809
           9       0.15      0.10      0.12      3862
          10       0.18      0.11      0.14      3771

    accuracy                           0.11     42240
   macro avg       0.12      0.11      0.11     42240
weighted avg       0.12      0.11      0.11     42240



#### Train the model using cross validation (3 fold) and display metrics.

In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

X = df.drop('Scale', axis = 1)
y = df['Scale']

pipeline = Pipeline(steps = [
  ("scaler", RobustScaler()),  
  ("model", KNeighborsClassifier(n_jobs = 4))
])

strat_k_fold = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 42)
scores = cross_validate(pipeline, X, y, cv = strat_k_fold, n_jobs = 4, return_train_score = True)

print("Train Accuracy: {:.2f} %".format(scores['train_score'].mean() * 100))
print("Test Accuracy: {:.2f} %".format(scores['test_score'].mean() * 100))


Train Accuracy: 36.73 %
Test Accuracy: 11.12 %
